In [1]:
import warnings

import pandas as pd
import numpy as np
import torch

from MLWrappers import PyTorchBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [2]:
# Definition of the PyTorch model

class Net(torch.nn.Module):
    """PyTorch neural network."""

    def __init__(self, n_features, n_classes):
        super().__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.fc1 = torch.nn.Linear(self.n_features, 64)
        self.fc99 = torch.nn.Linear(64, self.n_classes)
        self.dropout = torch.nn.Dropout(0.30)

    def forward(self, x):
        x = torch.nn.functional.tanh(self.fc1(x))
        x = self.dropout(x)
        out = torch.nn.functional.softmax(self.fc99(x), dim=1)
        return out

In [3]:
MODEL_NAME = 'nn_torch'
DS_NAME = 'adult'
DATA_FOLDER = f'./data/{DS_NAME}'

# We load the data used to train and test the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)
train_label = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_label.csv', skipinitialspace=True)
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)


In [4]:
# we load the target black box model using our wrapper

# For PyTorch we can load directly the jitted model
# This requires to convert the PyTorch model to a jitted one with torch.jit.script() and then .save()
target = PyTorchBlackBox(f'./models/{DS_NAME}_{MODEL_NAME}.pt')

# Otherwise we can load the PyTorch state discitonary
# This requires the use of the same class used to create the PyTorch network
target = PyTorchBlackBox(f'./models/{DS_NAME}_{MODEL_NAME}.pkl',
                         nn_class=Net(train_set.shape[1], len(np.unique(train_label))))

In [5]:
# We can define the parameters to be passed to the shadow models (in our case random forests)
shadow_forest_params = {'n_estimators': 100,
                        'max_depth': 10}

In [6]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target,
                       n_shadow_models=5)
label_only = LabelOnlyPrivacyAttack(target,
                                    n_shadow_models=1,
                                    shadow_model_type='rf',
                                    shadow_model_params=shadow_forest_params,
                                    n_noise_samples_fit=1000)
aloa = AloaPrivacyAttack(target,
                         n_shadow_models=1,
                         shadow_model_type='rf',
                         shadow_model_params=shadow_forest_params,
                         n_noise_samples_fit=1000,
                         shadow_test_size=0.3,
                         undersample_attack_dataset=True)
attacks = [mia, label_only, aloa]

In [7]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the list of attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [8]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set, save_folder = f'./results_{DS_NAME}_{MODEL_NAME}')

100%|██████████| 3622/3622 [00:20<00:00, 180.88it/s]


In [9]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)

100%|██████████| 24134/24134 [02:19<00:00, 173.54it/s]


{'mia_attack': {'classification_report': {'IN': {'precision': 0.8001101788287143, 'recall': 0.9779354638214119, 'f1-score': 0.880130520918308, 'support': 19307}, 'OUT': {'precision': 0.20522388059701493, 'recall': 0.022788481458462812, 'f1-score': 0.04102181614767853, 'support': 4827}, 'accuracy': 0.7868981519847518, 'macro avg': {'precision': 0.5026670297128646, 'recall': 0.5003619726399374, 'f1-score': 0.46057616853299327, 'support': 24134}, 'weighted avg': {'precision': 0.6811279893216117, 'recall': 0.7868981519847518, 'f1-score': 0.7123018262167323, 'support': 24134}}}, 'label_only_attack': {'classification_report': {'IN': {'precision': 0.8012942324482564, 'recall': 0.808100688869322, 'f1-score': 0.8046830677188097, 'support': 19307}, 'OUT': {'precision': 0.20544713703624276, 'recall': 0.19846695670188522, 'f1-score': 0.20189673340358272, 'support': 4827}, 'accuracy': 0.6861688903621447, 'macro avg': {'precision': 0.5033706847422496, 'recall': 0.5032838227856036, 'f1-score': 0.5032

In [10]:
results['mia_attack']['classification_report']

{'IN': {'precision': 0.8001101788287143,
  'recall': 0.9779354638214119,
  'f1-score': 0.880130520918308,
  'support': 19307},
 'OUT': {'precision': 0.20522388059701493,
  'recall': 0.022788481458462812,
  'f1-score': 0.04102181614767853,
  'support': 4827},
 'accuracy': 0.7868981519847518,
 'macro avg': {'precision': 0.5026670297128646,
  'recall': 0.5003619726399374,
  'f1-score': 0.46057616853299327,
  'support': 24134},
 'weighted avg': {'precision': 0.6811279893216117,
  'recall': 0.7868981519847518,
  'f1-score': 0.7123018262167323,
  'support': 24134}}

In [11]:
results['label_only_attack']['classification_report']

{'IN': {'precision': 0.8012942324482564,
  'recall': 0.808100688869322,
  'f1-score': 0.8046830677188097,
  'support': 19307},
 'OUT': {'precision': 0.20544713703624276,
  'recall': 0.19846695670188522,
  'f1-score': 0.20189673340358272,
  'support': 4827},
 'accuracy': 0.6861688903621447,
 'macro avg': {'precision': 0.5033706847422496,
  'recall': 0.5032838227856036,
  'f1-score': 0.5032899005611962,
  'support': 24134},
 'weighted avg': {'precision': 0.6821198755429033,
  'recall': 0.6861688903621446,
  'f1-score': 0.6841208055268978,
  'support': 24134}}

In [12]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.8000172436090874,
  'recall': 0.9612057802869426,
  'f1-score': 0.873235460191982,
  'support': 19307},
 'OUT': {'precision': 0.2006403415154749,
  'recall': 0.038947586492645535,
  'f1-score': 0.0652324774462179,
  'support': 4827},
 'accuracy': 0.776746498715505,
 'macro avg': {'precision': 0.5003287925622811,
  'recall': 0.500076683389794,
  'f1-score': 0.46923396881909996,
  'support': 24134},
 'weighted avg': {'precision': 0.6801368961156812,
  'recall': 0.776746498715505,
  'f1-score': 0.7116281676704852,
  'support': 24134}}